In [1]:
from langchain.document_loaders.text import TextLoader
from langchain.document_loaders.directory import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings

## Load the documents

In [2]:
import pathlib

p = pathlib.Path('/Users/manuel.mosquera/Documents/projects/mercado_libre_project')
list(p.glob('src/**/[!.]*.*'))

[PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/searchBar/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/searchBar/searchBar.test.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/searchBar/searchBar.styles.module.scss'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productsList/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productsList/productsList.styles.module.scss'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productsList/productsList.test.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productDetail/index.tsx'),
 PosixPath('/Users/manuel.mosquera/Documents/projects/mercado_libre_project/src/components/productDetail/productDetail.styles.m

In [3]:
directory_path = '/Users/manuel.mosquera/Documents/projects/mercado_libre_project'
loader = DirectoryLoader(directory_path, loader_cls=TextLoader, glob='src/**/[!.]*.*')
docs = loader.load()

In [4]:
len(docs)

24

In [5]:
print(docs[0].page_content)

import { useState, FC, SyntheticEvent } from "react";
import Image from "next/image";
import SearchIcon from "../../common/icons/search";

import styles from "./searchBar.styles.module.scss";

interface Props {
  placeholder?: string;
  onClick(val: string): void;
}

const SearchBar: FC<Props> = ({ placeholder, onClick }) => {
  const [search, setSearch] = useState("");

  const handleSubmit = (event: SyntheticEvent) => {
    event.preventDefault();
    onClick(search);
  };

  return (
    <header className={styles.container}>
      <Image
        data-testid="logo"
        src="/images/mercado-libre-logo.png"
        alt="Mercado libre logo"
        width="90"
        height="90"
      />
      <form
        className={styles.searchForm}
        onSubmit={handleSubmit}
        data-testid="form"
      >
        <input
          className={styles.searchInput}
          placeholder={placeholder}
          value={search}
          onChange={(e) => setSearch(e.target.value)}
          da

In [6]:
import re

preprocess_docs = [{'file': doc.page_content, 'file_path': re.match(directory_path + r'\/(.*)', doc.metadata['source']).group(1) } for doc in docs if doc.page_content is not None]

In [7]:
for p in preprocess_docs:
  print(p['file_path'])

src/components/searchBar/index.tsx
src/components/searchBar/searchBar.test.tsx
src/components/searchBar/searchBar.styles.module.scss
src/components/productsList/index.tsx
src/components/productsList/productsList.styles.module.scss
src/components/productsList/productsList.test.tsx
src/components/productDetail/index.tsx
src/components/productDetail/productDetail.styles.module.scss
src/components/productDetail/productDetail.test.tsx
src/components/breadcrumb/index.tsx
src/components/breadcrumb/breadcrumb.styles.module.scss
src/components/breadcrumb/breadcrumb.test.tsx
src/common/styles/variables.module.scss
src/common/styles/reset.styles.scss
src/common/icons/search.tsx
src/common/icons/types.ts
src/pages/index.tsx
src/pages/items.tsx
src/pages/_document.tsx
src/pages/_app.styles.module.scss
src/pages/_app.tsx
src/pages/api/items.tsx
src/pages/api/items/[id].tsx
src/pages/items/[id].tsx


## Generate a document description

In [8]:
from langchain.chat_models import ChatOpenAI

In [9]:
from langchain.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain

prompt_template = "What is the purpose of the following file in the project?\nGive me a detailed description of the file \nfile path: {file_path} \nfile content: {file}"

llm = ChatOpenAI(temperature=0.7)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)
files_description = llm_chain.apply(preprocess_docs)

In [10]:
len(files_description)

24

In [11]:
print(files_description[0]['text'])

The purpose of the file "index.tsx" in the "src/components/searchBar" directory is to define a React functional component for a search bar. 

The file imports the necessary dependencies, including "useState" from React, "Image" from Next.js, and a custom "SearchIcon" component. It also imports a SCSS module for styling.

The component accepts two prop types: "placeholder" (an optional string) and "onClick" (a function that takes a string parameter and returns void).

Inside the component, there is a local state variable "search" initialized with an empty string using the "useState" hook. The component also defines a "handleSubmit" function that calls the "onClick" prop with the current value of "search" when a form is submitted.

The component renders a header container with a Mercado Libre logo image and a search form. The form contains an input field for typing search queries, which is bound to the "search" state variable and updates it when the input value changes. There is also a s

## Create an embedding with the description

In [12]:
desciptions_to_embed =  [f'File path: {preprocess_docs[i]["file_path"]}\nFile description: {files_description[i]["text"]}' for i in range(len(files_description))]

In [13]:
print(desciptions_to_embed[0])

File path: src/components/searchBar/index.tsx
File description: The purpose of the file "index.tsx" in the "src/components/searchBar" directory is to define a React functional component for a search bar. 

The file imports the necessary dependencies, including "useState" from React, "Image" from Next.js, and a custom "SearchIcon" component. It also imports a SCSS module for styling.

The component accepts two prop types: "placeholder" (an optional string) and "onClick" (a function that takes a string parameter and returns void).

Inside the component, there is a local state variable "search" initialized with an empty string using the "useState" hook. The component also defines a "handleSubmit" function that calls the "onClick" prop with the current value of "search" when a form is submitted.

The component renders a header container with a Mercado Libre logo image and a search form. The form contains an input field for typing search queries, which is bound to the "search" state variabl

In [14]:
embeddings_model = OpenAIEmbeddings()

In [15]:
files_vectors = embeddings_model.embed_documents(desciptions_to_embed)

## Create the vector store

In [16]:
from langchain.vectorstores import Chroma


db = Chroma(persist_directory='../vector_db')

In [46]:
# db._client.reset()

True

In [17]:
import uuid

db._collection.add(embeddings=files_vectors, documents=[doc['file'] for doc in preprocess_docs], metadatas=[{'file_path': doc['file_path']} for doc in preprocess_docs], ids=[str(uuid.uuid1()) for _ in desciptions_to_embed])

In [18]:
db._collection.count()

24

In [19]:
db.persist()

## Create the steps to solve a task

In [20]:
task = "Add a pagination feature to the items list, listing 4 items per page."

In [27]:
embedded_task = embeddings_model.embed_query(task)
most_relevant_docs = db._collection.query(query_embeddings=[embedded_task], n_results=5)

In [28]:
most_relevant_docs

{'ids': [['f09f5fac-2012-11ee-b997-7af460f92e88',
   'f09f60ba-2012-11ee-b997-7af460f92e88',
   'f09f5e30-2012-11ee-b997-7af460f92e88',
   'f09f60e2-2012-11ee-b997-7af460f92e88',
   'f09f5e44-2012-11ee-b997-7af460f92e88']],
 'embeddings': None,
 'documents': [['import { FC, useCallback } from "react";\nimport { Response, Error } from "./api/items";\nimport Breadcrumb from "../components/breadcrumb";\nimport ProductsList from "../components/productsList";\nimport { server } from "../../config";\nimport { useRouter } from "next/router";\nimport Head from "next/head";\n\ninterface Props {\n  data: Response | Error;\n  search: string;\n}\n\nconst Items: FC<Props> = ({ data, search }) => {\n  const router = useRouter();\n\n  if ("error" in data) {\n    return <h1>There was an error</h1>;\n  }\n\n  const handleClick = useCallback(\n    (categoryName) => {\n      const selectedCategory = data.categories_filter.find(\n        (category) => category.name === categoryName\n      );\n      if (se

In [29]:
most_relevant_docs = '\n\n'.join([f'File_path: {most_relevant_docs["metadatas"][0][i]}\nFile content: {most_relevant_docs["documents"][0][i]}' for i in range(len(most_relevant_docs['documents'][0]))])

In [30]:
print(most_relevant_docs)

File_path: {'file_path': 'src/pages/items.tsx'}
File content: import { FC, useCallback } from "react";
import { Response, Error } from "./api/items";
import Breadcrumb from "../components/breadcrumb";
import ProductsList from "../components/productsList";
import { server } from "../../config";
import { useRouter } from "next/router";
import Head from "next/head";

interface Props {
  data: Response | Error;
  search: string;
}

const Items: FC<Props> = ({ data, search }) => {
  const router = useRouter();

  if ("error" in data) {
    return <h1>There was an error</h1>;
  }

  const handleClick = useCallback(
    (categoryName) => {
      const selectedCategory = data.categories_filter.find(
        (category) => category.name === categoryName
      );
      if (selectedCategory) {
        router.push(`/items?search=${search}&category=${selectedCategory.id}`);
      }
    },
    [data, router]
  );

  return (
    <>
      {search && (
        <Head>
          <title>{search} | Mercado

In [31]:
import openai
import os

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_API_KEY")

In [32]:
chat_plan = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": f"Give me a detailed description of how I can accomplish the following task: {task}. This is the content of some files of the project: {most_relevant_docs}"}])

In [33]:
print(chat_plan.choices[0].message.content)

To add a pagination feature to the items list, listing 4 items per page, you can follow these steps:

1. Modify the `Items` component in the `src/pages/items.tsx` file to include state variables for the current page and the total number of pages. Also, modify the `ProductsList` component to accept the current page as a prop.

```jsx
import { FC, useCallback, useState } from "react";
import { Response, Error } from "./api/items";
import Breadcrumb from "../components/breadcrumb";
import ProductsList from "../components/productsList";
import { server } from "../../config";
import { useRouter } from "next/router";
import Head from "next/head";

interface Props {
  data: Response | Error;
  search: string;
}

const Items: FC<Props> = ({ data, search }) => {
  const router = useRouter();
  const [currentPage, setCurrentPage] = useState(1);
  const itemsPerPage = 4;

  if ("error" in data) {
    return <h1>There was an error</h1>;
  }

  const handleClick = useCallback(
    (categoryName) =>